# Perform EDA on data captured in DB

In [1]:
import pandas as pd
# import pymysql

In [2]:
%run secrets.py # this will import all vars from this file (e.g. HOST)

In [3]:
# import Subreddit, Submission, & Comments tables

In [4]:
# create three DFs that join related Subreddits-Submissions-Comments

In [5]:
# pick a DF & perform some EDA: counts of Submissions & Comments; counts of authors; popular n-grams (1 to 3) used in titles & body; tf-idf for titles & bodies(?)